In [6]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

import altair as alt

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [7]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_shs.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Month_no']= df['datetime'].dt.month
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Month_no'].astype(str)
df.head(3)

,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,datetime,Month_no,Year_no,Period
<Ticker>,,,,,,,,,,
SHS,20090625,10.5327,11.7333,9.8233,10.9693,1828600,2009-06-25,6,2009,2009-6
SHS,20090626,11.3240,11.4605,10.8601,10.8601,1163400,2009-06-26,6,2009,2009-6
SHS,20090629,11.0238,11.4331,10.8601,10.9965,1534300,2009-06-29,6,2009,2009-6


In [8]:
df = df.rename(columns={'<CloseFixed>':'<Close>'})
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(15000)

In [9]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='SHS'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [10]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart', height= 800, width= 1200)

In [11]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
SHS,40.7,2022-03-16,3070030,2022-3,2022
SHS,41.3,2022-03-17,2507844,2022-3,2022
SHS,41.4,2022-03-18,5649801,2022-3,2022


In [12]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
SHS,10.8601,2009-06-26,1163400,2009-6,2009,-0.995506
SHS,10.9965,2009-06-29,1534300,2009-6,2009,1.255974
SHS,10.2871,2009-06-30,867200,2009-6,2009,-6.451144


In [13]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [14]:
df['Daily_return'].describe()

count    3179.000000
mean        0.095820
std         3.293453
min       -11.266611
25%        -1.724922
50%         0.000000
75%         1.818470
max        14.813171
Name: Daily_return, dtype: float64

In [15]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [16]:
df['<Close>'].describe()

count    3179.000000
mean        7.894366
std         9.201938
min         1.285000
25%         3.120700
50%         5.094000
75%         8.231900
max        55.500000
Name: <Close>, dtype: float64

In [17]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [18]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [19]:
df['Volumn'].head(30).describe()

count    30.000000
mean      0.787827
std       0.706404
min       0.049500
25%       0.425800
50%       0.579150
75%       0.940325
max       3.845100
Name: Volumn, dtype: float64

In [20]:
df['Volumn'].head(10).describe()

count    10.000000
mean      1.003080
std       0.489796
min       0.197000
25%       0.662375
50%       1.032250
75%       1.439700
max       1.581200
Name: Volumn, dtype: float64

In [21]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
SHS,10.8601,2009-06-26,1163400,2009-6,2009,-0.995506,1.1634
SHS,10.9965,2009-06-29,1534300,2009-6,2009,1.255974,1.5343
SHS,10.2871,2009-06-30,867200,2009-6,2009,-6.451144,0.8672
